# Base séquentielle des parcours de soins patients

Ce notebook a pour objectif de déterminer les flux de patients, c'est-à-dire visualiser les proportions de GHM et DP selon les hospitalisations successives.

In [1]:
#Import des librairies

import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import plotly.express as px
import numpy as np

In [2]:
%%capture capt
p_soins=pd.read_csv("./data/parcours_soins.csv")
patients=pd.read_csv("./data/grid_search/filter_20_10_5.csv")

In [3]:
p_soins=p_soins.rename(columns={"BEN_NIR_IDT": "CODE_PATIENT"})
p_soins=p_soins.set_index("CODE_PATIENT")
patients=patients.set_index("CODE_PATIENT")

In [4]:
nb_cluster = 1
found_max = False
while not found_max:
    try:
        patients['cluster'+str(nb_cluster)]
        nb_cluster += 1
    except:
        found_max=True

nb_cluster -= 1
patients['cluster'] = [1+np.argmax(patients.loc[:, 'cluster1':'cluster'+str(nb_cluster)].iloc[i]) for i in range(len(patients))]

In [5]:
p_soins["cluster"]=patients["cluster"]

/tmp/ipykernel_559/1368965147.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  p_soins["cluster"]=patients["cluster"]


In [6]:
p_soins=p_soins.reset_index()

In [7]:
p_soins.head()

,CODE_PATIENT,2008-01-15,2008-02-15,2008-03-15,2008-04-15,2008-05-15,2008-06-15,2008-07-15,2008-08-15,2008-09-15,...,2018-12-20,2018-12-21,2018-12-22,2018-12-24,2018-12-25,2018-12-26,2018-12-27,2018-12-28,2018-12-29,cluster
0,P0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
1,P6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
2,P8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,P13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,P16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4


## Création de la base séquentielle d'hospitalisations

### Pour l'ensemble de la population

In [14]:
p_soins_list=[]
for patient in tqdm(range(len(p_soins))):
    #ajout du décès
    if patients.iloc[patient].Mort:
        p_soins_list.append([p_soins.iloc[patient].CODE_PATIENT]+[patients.iloc[patient].cluster]+p_soins.iloc[patient].dropna().tolist()[1:-1]+['Décès'])
    else:
        p_soins_list.append([p_soins.iloc[patient].CODE_PATIENT]+[patients.iloc[patient].cluster]+p_soins.iloc[patient].dropna().tolist()[1:-1])

100%|██████████| 10051/10051 [00:50<00:00, 200.89it/s]


Dans la base séquentielle ci-dessous, les différentes colonnes correspondent aux hospitalisations successives. Ainsi, on trouve dans la 1ère colonne le GHM de la 1ère hospitalisation, pour chacun des patients suivis.

In [15]:
p_soins_df=pd.DataFrame(p_soins_list)
p_soins_df.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,P0,4,05K051,05M042,05M16T,05M09T,05M092,05C191,05M20Z,05M20Z,...,None,None,None,None,None,None,None,None,None,None
1,P6,4,02C05J,02C05J,05M093,04M132,05C222,23M103,04M053,04M24E,...,None,None,None,None,None,None,None,None,None,None
2,P8,2,11M041,06C194,Décès,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,P13,2,05M04T,05K051,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,P16,4,08C11V,23M15Z,05K102,05M08T,05K102,05C191,23M101,05K191,...,None,None,None,None,None,None,None,None,None,None


### Fusion des opérations de la cataracte en un seul GHM

+ [02C05J, 02C05J] --> [02C05J]
+ [02C051, 02C051] --> [02C051]

In [35]:
p_soins_new=[]
for patient in tqdm(range(len(p_soins_df))):
    p_soin = p_soins_df.iloc[patient].dropna().to_list()
    while p_soin.count('02C05J')>1:
        p_soin.remove('02C05J')
    p_soins_new.append(p_soin)

100%|██████████| 10051/10051 [00:03<00:00, 2659.82it/s]


In [38]:
p_soins_df=pd.DataFrame(p_soins_new)

In [39]:
p_soins_new=[]
for patient in tqdm(range(len(p_soins_df))):
    p_soin = p_soins_df.iloc[patient].dropna().to_list()
    while p_soin.count('02C051')>1:
        p_soin.remove('02C051')
    p_soins_new.append(p_soin)

100%|██████████| 10051/10051 [00:03<00:00, 2769.55it/s]


In [40]:
p_soins_df.to_csv("./data/p_soins.csv")

### Pour les patients avec une hospitalisation pour un GHM '05M09x', en considérant cette hospitalisation comme début de parcours 

In [43]:
p_soins_df.iloc[0].dropna()

0         P0
1          4
2     05K051
3     05M042
4     05M16T
5     05M09T
6     05M092
7     05C191
8     05M20Z
9     05M20Z
10    05C191
Name: 0, dtype: object

In [47]:
p_soins_IC=[]
for patient in tqdm(range(len(p_soins_df))):
    parcours=p_soins_df.iloc[patient].dropna()
    #Si on trouve un GHM de type '05M09x' dans le parcours de soin du patient
    if parcours.eq('05M091').any() or parcours.eq('05M092').any() or parcours.eq('05M093').any() or parcours.eq('05M094').any() or parcours.eq('05M09T').any():
        start=0
        #On cherche la 1ère hospitalisation pour IC, pour regarder le parcours de soins à partir de celle-ci
        while parcours.tolist()[start] not in ['05M091', '05M092', '05M093', '05M094','05M09T']:
            start+=1
        p_soins_IC.append([parcours[0]]+[parcours[1]]+parcours.tolist()[start:])

100%|██████████| 10051/10051 [00:10<00:00, 941.31it/s]


In [48]:
p_soins_IC_df=pd.DataFrame(p_soins_IC)
p_soins_IC_df.head()

,0,1,2,3,4,5,6,7,8,9,...,115,116,117,118,119,120,121,122,123,124
0,P0,4,05M09T,05M092,05C191,05M20Z,05M20Z,05C191,None,None,...,None,None,None,None,None,None,None,None,None,None
1,P6,4,05M093,04M132,05C222,23M103,04M053,04M24E,Décès,None,...,None,None,None,None,None,None,None,None,None,None
2,P17,5,05M093,04M143,11M044,04M053,09M063,05K251,09M062,09M063,...,None,None,None,None,None,None,None,None,None,None
3,P24,1,05M091,05M171,05M13T,05M13T,05M13T,05K061,05M13T,05M13T,...,None,None,None,None,None,None,None,None,None,None
4,P25,4,05M091,23K02Z,09C081,09C082,19M02T,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [50]:
p_soins_IC_df.describe()

,1
count,4637.000000
mean,2.665301
std,1.237757
min,1.000000
25%,2.000000
50%,2.000000
75%,4.000000
max,5.000000


In [49]:
p_soins_IC_df.to_csv("./data/p_soins_IC.csv")